# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
#Load CSV file
cities_df = pd.read_csv('../WeatherPy/WeatherPy_Output.csv')

#Print
cities_df

,Unnamed: 0,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Bluff,-46.6000,168.3333,73.99,54,100,15.01,NZ,1611694499
1,1,Gerāsh,27.6652,54.1371,46.40,46,0,1.99,IR,1611694274
2,2,Carnarvon,-24.8667,113.6333,69.80,73,8,20.71,AU,1611694211
3,3,Arraial do Cabo,-22.9661,-42.0278,84.20,65,0,17.27,BR,1611694505
4,4,Ambilobe,-13.2000,49.0500,75.81,90,78,1.12,MG,1611694334
...,...,...,...,...,...,...,...,...,...,...
538,538,Dalby,-27.1833,151.2667,77.00,62,0,1.99,AU,1611695139
539,539,Palatka,29.6486,-81.6376,79.00,69,81,8.05,US,1611695062
540,540,Rurrenabaque,-14.4413,-67.5278,80.60,94,75,6.04,BO,1611695141
541,541,Anadyr,64.7500,177.4833,-14.01,76,0,6.71,RU,1611695142


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [69]:
# Store latitude and longitude in locations and humidity as the weights
heat_locations = cities_df[["Latitude", "Longitude"]]

humidity = cities_df["Humidity"]

figure_layout = {
    'width': '700px',
    'height': '400px'
}

# Plot Heatmap
fig = gmaps.figure(layout=figure_layout)

# Create heat layer
heat_layer = gmaps.heatmap_layer(heat_locations, weights=humidity)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='400px', width='700px'))

In [70]:
#Customize the layout
heat_layer.max_intensity=100
heat_layer.point_radius=3
heat_layer.dissipating=False

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [24]:
#Narrow down the vacation spot conditions
temp_filtered_df = cities_df.loc[(cities_df['Max Temperature']) > 80, :]
vacation_df = temp_filtered_df.loc[(temp_filtered_df['Cloudiness']) <= 0, :]

#Print
vacation_df

,Unnamed: 0,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
3,3,Arraial do Cabo,-22.9661,-42.0278,84.20,65,0,17.27,BR,1611694505
35,35,Lagunillas,10.1301,-71.2595,84.79,61,0,9.26,VE,1611694542
105,105,Neuquén,-38.9516,-68.0591,84.20,21,0,9.22,AR,1611694623
116,116,Barra de Tecoanapa,16.5167,-98.7500,83.16,59,0,11.25,MX,1611694636
185,185,General Roca,-39.0333,-67.5833,84.20,21,0,9.22,AR,1611694718
254,254,Talara,-4.5772,-81.2719,86.00,51,0,17.27,PE,1611694797
268,268,São João da Barra,-21.6403,-41.0511,84.20,58,0,23.02,BR,1611694824
506,506,Te Karaka,-38.4654,177.8679,84.83,37,0,2.84,NZ,1611695102
537,537,Touros,-5.1989,-35.4608,82.40,69,0,16.11,BR,1611695138


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [29]:
hotel_df = vacation_df[['City', 'Country', 'Latitude', 'Longitude']]

hotel_df["Hotel Name"] = " "

hotel_df

<ipython-input-29-98073b8c11b4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = " "


,City,Country,Latitude,Longitude,Hotel Name
3,Arraial do Cabo,BR,-22.9661,-42.0278,
35,Lagunillas,VE,10.1301,-71.2595,
105,Neuquén,AR,-38.9516,-68.0591,
116,Barra de Tecoanapa,MX,16.5167,-98.7500,
185,General Roca,AR,-39.0333,-67.5833,
254,Talara,PE,-4.5772,-81.2719,
268,São João da Barra,BR,-21.6403,-41.0511,
506,Te Karaka,NZ,-38.4654,177.8679,
537,Touros,BR,-5.1989,-35.4608,


In [39]:
# find the closest restaurant of each type to coordinates
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
}
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    city = row['City']
    
    # get location from df
    lat = row['Latitude']
    lng = row['Longitude']

    # add location to params dict
    params['location'] = f"{lat},{lng}"
    
    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    #print(results)
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 3: Arraial do Cabo.
Closest hotel is Pousada Porto Praia.
------------
Retrieving Results for Index 35: Lagunillas.


C:\Users\wazz_\Anaconda3.1\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Closest hotel is Madre de Kirvez Castillo.
------------
Retrieving Results for Index 105: Neuquén.
Closest hotel is Hotel del Comahue.
------------
Retrieving Results for Index 116: Barra de Tecoanapa.
Closest hotel is Hotel Manglares.
------------
Retrieving Results for Index 185: General Roca.
Closest hotel is Hotel El Recreo.
------------
Retrieving Results for Index 254: Talara.
Closest hotel is Hostal Oro Negro.
------------
Retrieving Results for Index 268: São João da Barra.
Closest hotel is Pousada Porto De Canoas.
------------
Retrieving Results for Index 506: Te Karaka.
Missing field/result... skipping.
------------
Retrieving Results for Index 537: Touros.
Closest hotel is INN NEW HORIZON.
------------


In [40]:
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
3,Arraial do Cabo,BR,-22.9661,-42.0278,Pousada Porto Praia
35,Lagunillas,VE,10.1301,-71.2595,Madre de Kirvez Castillo
105,Neuquén,AR,-38.9516,-68.0591,Hotel del Comahue
116,Barra de Tecoanapa,MX,16.5167,-98.7500,Hotel Manglares
185,General Roca,AR,-39.0333,-67.5833,Hotel El Recreo
254,Talara,PE,-4.5772,-81.2719,Hostal Oro Negro
268,São João da Barra,BR,-21.6403,-41.0511,Pousada Porto De Canoas
506,Te Karaka,NZ,-38.4654,177.8679,
537,Touros,BR,-5.1989,-35.4608,INN NEW HORIZON


In [71]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [72]:
# Add marker layer ontop of heat map
figure_layout = {
    'width': '700px',
    'height': '400px'
}

# Plot Heatmap
fig = gmaps.figure(layout=figure_layout)

# Create heat layer
heat_layer = gmaps.heatmap_layer(heat_locations, weights=humidity)
hotel_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Add layer
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)

#Customize the layout
heat_layer.max_intensity=100
heat_layer.point_radius=3
heat_layer.dissipating=False

# Display figure
fig

Figure(layout=FigureLayout(height='400px', width='700px'))